# Projeto H₂ no Google ColabNotebook para EDA, KPIs operacionais (HESS), índice de atratividade de hubs de H₂ e simulações de substituição de H₂ cinza por verde.

In [ ]:
!pip -q install pandas numpy seaborn plotly scikit-learn

In [ ]:
import pandas as pdimport numpy as npimport seaborn as snsimport plotly.express as pxfrom sklearn.preprocessing import MinMaxScalerimport os

In [ ]:
from google.colab import drivedrive.mount('/content/drive')

## PathsDefina a base para `MyDrive/h2_project` com a estrutura descrita no README.

In [ ]:
BASE = '/content/drive/MyDrive/h2_project'

## Carregar Datasets

In [ ]:
hess_path = os.path.join(BASE,'hybrid_energy_storage_dataset','hybrid_energy_storage.csv')rh_path = os.path.join(BASE,'renewable_hydrogen_dataset','renewable_hydrogen_dataset_2535.csv')hess = pd.read_csv(hess_path)rh = pd.read_csv(rh_path)

## EDA HESS

In [ ]:
hess_stats = hess.describe().Thess_missing = hess.isna().sum()hess_balance = hess['Power_Supplied_kW'] - hess['Load_Demand_kW'] - hess['Power_Loss_kW']hess_balance_stats = pd.Series({'min': hess_balance.min(), 'max': hess_balance.max(), 'mean': hess_balance.mean()})print(hess_stats.to_string())print(hess_missing.to_string())print(hess_balance_stats.to_string())

## EDA Renewable Hydrogen 2535

In [ ]:
rh_stats = rh.describe().Trh_missing = rh.isna().sum()print(rh_stats.to_string())print(rh_missing.to_string())

## Indicadores Externos

In [ ]:
emissions = pd.read_csv(os.path.join(BASE,'external','emissions_per_capita.csv'))ren_share = pd.read_csv(os.path.join(BASE,'external','renewables_share.csv'))gdp_pc = pd.read_csv(os.path.join(BASE,'external','gdp_per_capita.csv'))elec_access = pd.read_csv(os.path.join(BASE,'external','electricity_access.csv'))city_country = pd.read_csv(os.path.join(BASE,'external','city_to_country.csv'))

## Índice de Atratividade de Hubs de H₂

In [ ]:
df = rh.merge(city_country, on='City') \       .merge(emissions, on='Country') \       .merge(ren_share, on='Country') \       .merge(gdp_pc, on='Country') \       .merge(elec_access, on='Country')df['Renewables_share_inv'] = 1 - df['Renewables_share']Xcols = ['Hydrogen_Production_kg/day','Feasibility_Score','Electrolyzer_Efficiency_%','System_Efficiency_%','Emissions_per_capita','Renewables_share_inv','GDP_per_capita','Electricity_access']scaler = MinMaxScaler()X = scaler.fit_transform(df[Xcols])w = np.array([0.25,0.15,0.10,0.10,0.20,0.10,0.05,0.05])df['H2_hub_attractiveness'] = X.dot(w)ranking = df[['City','Country','H2_hub_attractiveness']].sort_values('H2_hub_attractiveness', ascending=False).head(50)print(ranking.to_string(index=False))

## Substituição H₂ Cinza → Verde

In [ ]:
cons = pd.read_csv(os.path.join(BASE,'external','h2_consumption.csv'))em_factor_grey = 10.0em_factor_green = 0.5sub = 0.5cons['CO2_reduction_tons'] = (cons['H2_grey_kg_per_year']*sub*(em_factor_grey - em_factor_green))/1000print(cons.sort_values('CO2_reduction_tons', ascending=False).head(50).to_string(index=False))

## Exportar Resultados

In [ ]:
out_dir = os.path.join(BASE,'outputs')os.makedirs(out_dir, exist_ok=True)ranking.to_csv(os.path.join(out_dir,'h2_hub_ranking.csv'), index=False)cons.to_csv(os.path.join(out_dir,'h2_substitution_co2_reduction.csv'), index=False)

## Visualização

In [ ]:
fig = px.scatter(df, x='Solar_Irradiance_kWh/m²/day', y='Hydrogen_Production_kg/day', color='H2_hub_attractiveness', hover_data=['City','Country'])fig.show()